In [1]:
%%time
import sys
sys.path.append('../python/')

from trectools import TrecRun, TrecQrel
import pandas as pd
from tqdm import tqdm
from run_file_processing import load_all_runs, normalize_run
from evaluation_util import __adjust_qrels_to_pool
from probability_estimation import load_pool_task, CountProbabilityEstimator, RunIndependentCountProbabilityEstimator, PoissonEstimator

RUNS = list(load_all_runs('../resources/processed/normalized-runs/trec-system-runs/trec13/robust').keys())

class PredictAlways0:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:1, 1:0, 2:0, 3:0}

class PredictAlways1:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:0, 1:1, 2:0, 3:0}

def eval_on_x(run, qrels_incomplete, qrels_complete):
    run = normalize_run(run, 10).run_data
    ret = []
    
    estimators = [('always-0', PredictAlways0()),
                  ('cnt', CountProbabilityEstimator()),
                  ('run-indep', RunIndependentCountProbabilityEstimator()),
                  ('poisson', PoissonEstimator()),
                  ('always-1', PredictAlways1())
                 ]
    
    for topic in tqdm(run['query'].unique()):
        run_for_topic = TrecRun()
        
        run_for_topic.run_data = pd.merge(run, qrels_incomplete[["query","docid","rel"]], how="left")
        run_for_topic.run_data = run_for_topic.run_data[run_for_topic.run_data['query'] == topic]
        
        df = run_for_topic.run_data.copy()
        df = df[df["rel"].isnull()]
        del df['rel']
        df = pd.merge(df, qrels_complete[["query","docid","rel"]], how="left")
        df = df[~df["rel"].isnull()]
        
        if len(df) == 0:
            continue
        
        expected = df['rel'].value_counts().to_dict()
        sum_expected = sum(expected.values())
        expected = {i: expected.get(i, 0)/sum_expected for i in [0, 1, 2, 3]}

        tmp = {'system': df.iloc[0]['system'], 'query': df.iloc[0]['query'], 'expected': expected}
        for estimator_name, estimator in estimators:
            tmp[estimator_name] = estimator.estimate_probabilities(run_for_topic, qrels)

        ret += [tmp]

    return pd.DataFrame(ret)

Load runs: 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [00:05<00:00, 18.83it/s]

CPU times: user 6.14 s, sys: 791 ms, total: 6.93 s
Wall time: 6.57 s


In [2]:
tmp_ret = []

for run_name in RUNS:
    run = normalize_run(TrecRun(run_name), 10)
    qrels = TrecQrel('../resources/unprocessed/topics-and-qrels/qrels.robust04.txt')
    task = {
        'working_directory': '../resources/',
        'trec_identifier': 'trec-system-runs-trec13-robust',
        'run': run_name.replace('../resources', 'src/main/resources')
    }
    pool = load_pool_task(task)
    pool = {k:v for k,v in pool.items() if 'depth-10-pool-incomplete-for' in k}
    assert len(pool) == 1
    pool = pool[list(pool.keys())[0]]
    
    qrels_incomplete = __adjust_qrels_to_pool(qrels, pool)
    
    tmp_ret += [eval_on_x(run, qrels_incomplete.qrels_data.copy(), qrels.qrels_data.copy())]

    if len(tmp_ret) > 2:
        break
#tmp_ret = pd.concat(tmp_ret)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [02:14<00:00,  1.86it/s]


In [21]:




df = tmp_ret[0].copy()
for field in ['always-0', 'cnt', 'run-indep', 'poisson']:
    df[f'loss-{field}'] = df.apply(lambda i: jensenshannon_loss(i, field), axis=1)

df

,system,query,expected,always-0,cnt,run-indep,poisson,always-1,loss-always-0,loss-cnt,loss-run-indep,loss-poisson
0,apl04rsTDNw5,301,"{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 9.997000899730082e-05, 1: 0.99970008997300...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 2.7180100138671515e-32, 1: 1.0, 2: 2.71801...","{0: 0, 1: 1, 2: 0, 3: 0}",0.833,0.010,0.767,0.000
1,apl04rsTDNw5,306,"{0: 0.5, 1: 0.5, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.8748250349930015, 1: 0.1249750049990002,...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 0.9999837197954541, 1: 1.6280204545874084e...","{0: 0, 1: 1, 2: 0, 3: 0}",0.465,0.294,0.375,0.464
2,apl04rsTDNw5,307,"{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.7498500299940012, 1: 0.2499500099980004,...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 0.9932609152250002, 1: 0.00673908477499984...","{0: 0, 1: 1, 2: 0, 3: 0}",0.833,0.617,0.767,0.820
3,apl04rsTDNw5,311,"{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.11108889333244462, 1: 0.888711146659557,...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 1.297372370556767e-07, 1: 0.99999987026276...","{0: 0, 1: 1, 2: 0, 3: 0}",0.833,0.201,0.767,0.000
4,apl04rsTDNw5,313,"{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 9.997000899730082e-05, 1: 0.99970008997300...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 2.718010013867145e-16, 1: 0.99999999999999...","{0: 0, 1: 1, 2: 0, 3: 0}",0.833,0.010,0.767,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
97,apl04rsTDNw5,686,"{0: 1.0, 1: 0.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.7998400319936013, 1: 0.19996000799840033...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 0.9930878548582619, 1: 0.00691214514173723...","{0: 0, 1: 1, 2: 0, 3: 0}",0.000,0.274,0.141,0.049
98,apl04rsTDNw5,692,"{0: 1.0, 1: 0.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.3749250149970006, 1: 9.998000399920017e-...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 0.05628495664862138, 1: 2.9371932040709627...","{0: 0, 1: 1, 2: 0, 3: 0}",0.000,0.539,0.141,0.764
99,apl04rsTDNw5,698,"{0: 1.0, 1: 0.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.856971462850287, 1: 9.998000399920017e-0...","{0: 0.943992174752532, 1: 0.05259741839951709,...","{0: 0.9997422335837128, 1: 4.994596752892839e-...","{0: 0, 1: 1, 2: 0, 3: 0}",0.000,0.229,0.141,0.009
100,apl04rsTDNw5,699,"{0: 1.0, 1: 0.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.5554444666622231, 1: 0.4443555733297785,...","{0: 0.9439921747525322, 1: 0.0525974183995171,...","{0: 0.7319633354719777, 1: 0.2680366645280222,...","{0: 0, 1: 1, 2: 0, 3: 0}",0.000,0.432,0.141,0.321


In [22]:
df['loss-cnt'].mean()

0.35159803921568633

In [23]:
df['loss-always-0'].mean()

0.1758627450980392

In [19]:
df['loss-run-indep'].mean()

0.19124000000000002

In [20]:
df['loss-poisson'].mean()

0.20808

In [62]:


jensenshannon(np.array([0.8, 0.2, 0, 0]), np.array([0.0,0.0,0,1]))

0.8325546111576977

In [ ]:
print('sdaas')

In [ ]:
import sys
sys.path.append('../python/')

from trectools import TrecRun, TrecQrel
import pandas as pd
from tqdm import tqdm
from run_file_processing import load_all_runs, normalize_run
from evaluation_util import __adjust_qrels_to_pool
from probability_estimation import load_pool_task, CountProbabilityEstimator, RunIndependentCountProbabilityEstimator, PoissonEstimator
import json
from io import StringIO
entries = []
import pandas as pd
from trectools import TrecRun
from copy import deepcopy

class PredictAlways0:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:1, 1:0, 2:0, 3:0}

class PredictAlways1:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:0, 1:1, 2:0, 3:0}


class PredictAlways005:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:0.95, 1:0.05, 2:0, 3:0}

    
class PredictAlways01:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:0.9, 1:0.1, 2:0, 3:0}

class PredictAlways015:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:0.85, 1:0.15, 2:0, 3:0}
    
def eval_line(line):
    estimators = [('always-0', PredictAlways0()),
                  ('cnt', CountProbabilityEstimator()),
                  ('run-indep', RunIndependentCountProbabilityEstimator()),
                  ('poisson', PoissonEstimator()),
                  ('always-1', PredictAlways1()),
                  ('always-01', PredictAlways01()),
                  ('always-015', PredictAlways015()),
                  ('always-005', PredictAlways005()),
                 ]
    
    run_for_topic = line['run'].run_data
    qrels_incomplete = TrecQrel()
    qrels_incomplete.qrels_data = line['qrels_incomplete']
    qrels_complete = line['qrels_complete']
    
    #print(run_for_topic)
    str(qrels_incomplete.qrels_data[["query","docid","rel"]])
    str(run_for_topic.columns)
    
    run_for_topic = pd.merge(run_for_topic.copy(), qrels_incomplete.qrels_data[["query","docid","rel"]], how="left")
        
    df = run_for_topic.copy()
    df = df[df["rel"].isnull()]
    del df['rel']
    df = pd.merge(df, qrels_complete[["query","docid","rel"]], how="left")
    df = df[~df["rel"].isnull()]
        
    if len(df) == 0:
        return None
        
    expected = df['rel'].value_counts().to_dict()
    sum_expected = sum(expected.values())
    expected = {i: expected.get(i, 0)/sum_expected for i in [0, 1, 2, 3]}
    
    ret = {'system': df.iloc[0]['system'], 'query': df.iloc[0]['query'], 'expected': expected}
    
    run = TrecRun()
    run.run_data = run_for_topic
    
    for estimator_name, estimator in estimators:
        ret[estimator_name] = estimator.estimate_probabilities(run, qrels_incomplete)

    return ret

df = []
df_raw = []


for i in tqdm(list(open('probability-estimation-playground.jsonl', 'r'))[:1000]):
    try:
        i = json.loads(i)
        i['qrels_incomplete'] = pd.read_json(StringIO(i['qrels_incomplete']), lines=True)
        i['qrels_complete'] = pd.read_json(StringIO(i['qrels_complete']), lines=True)
        run = TrecRun()
        run.run_data = pd.read_json(StringIO(i['run']), lines=True)
        i['run'] = run
        
        df_raw += [deepcopy(i)]
        
        i = eval_line(i)
        df += [i]
        
        if len(df) > 10000:
            break
    except:
        pass

df = pd.DataFrame([i for i in df if i])
df

In [4]:
from scipy.spatial.distance import jensenshannon

def jensenshannon_loss(data, field):
    fields = [0, 1, 2, 3]
    y_true = [data['expected'].get(i, 0) for i in fields]
    y_pred = [data[field].get(i, 0) for i in fields]
    
    return round(jensenshannon(y_true, y_pred), 3)

def mean(data, field):
    ret = 0
    
    for k in [0, 1, 2, 3]:
        ret += k * data[field].get(k, 0)
    
    return ret

for field in ['expected', 'always-0', 'cnt', 'run-indep', 'poisson', 'always-1', 'always-01', 'always-015', 'always-005']:
    df[f'loss-{field}'] = df.apply(lambda i: jensenshannon_loss(i, field), axis=1)
    df[f'mean-{field}'] = df.apply(lambda i: mean(i, field), axis=1)

df.head(2)

,system,query,expected,always-0,cnt,run-indep,poisson,always-1,always-01,always-015,...,loss-poisson,mean-poisson,loss-always-1,mean-always-1,loss-always-01,mean-always-01,loss-always-015,mean-always-015,loss-always-005,mean-always-005
0,apl04rsTDNw5,301,"{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 9.997000899730082e-05, 1: 0.99970008997300...","{0: 0.6230637930384938, 1: 0.3767362469535079,...","{0: 2.7180100138671515e-32, 1: 1.0, 2: 2.71801...","{0: 0, 1: 1, 2: 0, 3: 0}","{0: 0.9, 1: 0.1, 2: 0, 3: 0}","{0: 0.85, 1: 0.15, 2: 0, 3: 0}",...,0.000,1.000000,0.000,1,0.725,0.1,0.686,0.15,0.770,0.05
1,apl04rsTDNw5,306,"{0: 0.5, 1: 0.5, 2: 0.0, 3: 0.0}","{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.8748250349930015, 1: 0.1249750049990002,...","{0: 0.5967392908852784, 1: 0.4030607491067231,...","{0: 0.9999837197954541, 1: 1.6280204545874084e...","{0: 0, 1: 1, 2: 0, 3: 0}","{0: 0.9, 1: 0.1, 2: 0, 3: 0}","{0: 0.85, 1: 0.15, 2: 0, 3: 0}",...,0.464,0.000016,0.465,1,0.319,0.1,0.270,0.15,0.377,0.05


In [7]:
{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}


{0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0}

In [6]:
df_eval = []

for field in ['expected', 'always-0', 'cnt', 'run-indep', 'poisson', 'always-1', 'always-01', 'always-015', 'always-005']:
    df_eval += [{'Method': field, 'JS Loss (Mean)': df[f'loss-{field}'].mean(), 'Mean Pred.': df[f'mean-{field}'].mean()}]

df_eval = pd.DataFrame(df_eval)
df_eval

,Method,JS Loss (Mean),Mean Pred.
0,expected,0.000000,0.155271
1,always-0,0.124644,0.000000
2,cnt,0.352396,0.480507
3,run-indep,0.274897,0.222130
4,poisson,0.291983,0.434211
5,always-1,0.724120,1.000000
6,always-01,0.258929,0.100000
7,always-015,0.288493,0.150000
8,always-005,0.220803,0.050000


In [ ]:
P(Relevant|Ungejudged)

P(Rel=0|Ungejudged)
P(Rel=1|Ungejudged)
P(Rel=2|Ungejudged)